# Installation


## Setup Windows and WSL to share the USB camera
Steps:
1. Install usbipd on Windows
2. Look for the camera's BUSID and attach it to WSL
3. Install the necessary packages in WSL to use the camera (v4l2 video for Linux 2) (install the toolkit v4l-utils)



### Windows setup

#### Install usbipd on Windows
Open a PowerShell with admin privileges and run the following command to install usbipd:

``` PowerShell
$ winget install dorssel.usbipd-win
```

#### Look for the camera's BUSID and attach it to WSL
Identify the BUSID of the camera in Windows:
``` PowerShell
$ usbipd list
```

#### Bind the camera to WSL:
In the list of devices the BUSID will look like 1-2 or similar. Write it instead of `<BUSID>` in the following command to attach the camera to WSL:
``` PowerShell
$ usbipd bind --busid <BUSID>
```
It will say to usbipd server to take control of the USB device, so the device is reserved abd ready to be exported through the network.

#### Attach the reserved USB device to WSL:
Builds a virtual bridge so WSL will see the USB device as if it were directly connected to the Linux system. 
``` PowerShell
$ usbipd attach --wsl --busid <BUSID>
```

#### Note
If you restart your computer, you will need to repeat the last two steps to reattach the camera to WSL. You can create a script to automate this process if needed.  
If you are done using it with Linux entirely, you should unbind it. This tells Windows it no longer needs to reserve the device for usbipd.

```PowerShell
$ usbipd unbind --busid <BUSID>
```

### WSL setup

#### Update and upgrade  
Update and upgrade. -y allows to skip the confirmation prompt.
``` zsh
sudo apt update
sudo apt upgrade -y
```

#### Install libraries to communicate with windows usbipd server
+ linux-tools-generic: Installa usbip binary. It is the client that receives the device from Windows.  
+ hwdata: It is a database to know the hardware IDs (VendorID/ProductID). It helps that when you write lsusb, Linux tells you "Logitech Webcam" instead of a cryptic numeric code.
+ update-alternatives: Set the Shortcut from wherever the kernel installs the `usbip` binary to ``/usr/local/bin/usbip``, so it is easier to call it from the terminal. It also allows to update the shortcut if the kernel updates and installs a new version of usbip in a different location.

``` zsh
sudo apt install linux-tools-generic hwdata
sudo update-alternatives --install /usr/local/bin/usbip usbip `ls /usr/lib/linux-tools/*/usbip | tail -n1` 20
```

#### Install the video packages for Linux

``` zsh
sudo apt install v4l-utils
```

#### Check if the camera is recognized in Linux
``` zsh
$ lsusb
```
You should see the camera listed in the output.  

Or: 

``` zsh
$ v4l2-ctl --list-devices
```
Output:
``` output
Streaming Webcams: Streaming We (usb-vhci_hcd.0-1):
        /dev/video0
        /dev/video1
        /dev/media0
```
Where:
- /dev/video0: This is the primary video stream
- /dev/video1: This is usually a metadata/output node. Modern Linux drivers often create a second node for things like camera controls or specific hardware formats.
- /dev/media0: This is a Media Controller device. It allows you to see the internal topology of the camera (how the sensor connects to the processing unit).


### Other useful commands
To see what controls the camera has (brightness, contrast, etc.) and their current values:
``` zsh
v4l2-ctl --device=/dev/video0 --list-ctrls
```

#### OpenCv
To set the autofocus:
``` Python
# Ensure Auto Focus is OFF (0)
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0) 
# Set value between 0 and 1023
cap.set(cv2.CAP_PROP_FOCUS, 500)
```

In [2]:
import cv2

cap = cv2.VideoCapture(0)

def on_change(val):
    cap.set(cv2.CAP_PROP_FOCUS, val)

cv2.namedWindow('Camera Settings')
# Create a slider for focus (0-1023)
cv2.createTrackbar('Focus', 'Camera Settings', 439, 1023, on_change)

# Disable Auto Focus first
cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)

while True:
    ret, frame = cap.read()
    if not ret: break
    
    cv2.imshow('Camera Settings', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Installing ROS2 in WSL



Depending on your ubuntu version you may install different versions of Ros2
[Ros2 webpage](https://www.ros.org/blog/getting-started/)  
It is recommended to install the deb packages:
[jazzy Ros2 Debpackage](https://docs.ros.org/en/jazzy/Installation/Ubuntu-Install-Debs.html )

Summary of packages to install

| Objetivo | Comando de Instalación | Qué instala exactamente |
|---|---|---|
| Base del Sistema | `sudo apt install ros-jazzy-desktop` | El núcleo de ROS 2, librerías de comunicación, RViz2, Gazebo y herramientas de desarrollo básico. |
| Cámara Web (USB) | `sudo apt install ros-jazzy-usb-cam` | El nodo estándar para leer cámaras por V4L2 (USB) y publicar imágenes en tópicos de ROS. |
| Cámara IP (RTSP) | `sudo apt install ros-jazzy-v4l2-camera` | Un driver flexible que permite conectar streams de video (incluyendo algunas cámaras IP) al ecosistema ROS. |
| Foxglove Studio | `sudo apt install ros-jazzy-foxglove-bridge` | El servidor WebSocket que traduce los datos internos de ROS a un formato que Foxglove puede graficar. |
| CV Bridge | `sudo apt install ros-jazzy-cv-bridge` | Permite la conversión entre imágenes de ROS y OpenCV, facilitando el procesamiento de imágenes en ROS. |

### Foxglove


[Foxglove Studio](https://docs.ros.org/en/jazzy/How-To-Guides/Visualizing-ROS-2-Data-With-Foxglove-Studio.html)
Install and make an account in FoxGlove.  

``` bash
$ sudo apt install ros-$ROS_DISTRO-foxglove-bridge
```
Estos comandos se utilizan para instalar y ejecutar el puente que permite que ROS 2 hable con aplicaciones externas (Foxglove Studio) vía WebSockets.

``` bash
$ ros2 launch foxglove_bridge foxglove_bridge_launch.xml port:=8765
```
- ros2 launch: Inicia un archivo de lanzamiento que arranca varios nodos o configuraciones a la vez.

- foxglove_bridge_launch.xml: El archivo que contiene las instrucciones para abrir el servidor del puente.

- port:=8765: Un argumento que define el puerto específico (8765) por el cual el puente escuchará conexiones.